## Homework 4- Task 3: Air Quality Index Data + False Color Image
Author: Hazel Vaquero

Repository link: https://github.com/hazelvaq/eds220-mpc.git 

## About


### Purpose

This notebook is a compilation of Homework 2-Task 2: Air Quality Index Data and Homework 4-Task 2: False color image. Our purpose is to analyze the affects the Thomas Fire had on Santa Barbara in 2017. We will visualize the fire scar of the fire and generate a time series displaying the fires impact on the Air Quality Index(AQI).

The notebook is divided in the following sections:
- Highlight of Analysis
- Dataset Description
- References to dataset
- Analysis/Visualization:
    - Importing data
    - Geographical context of data
    - Data Exploration
    - Analysis 
    - Final Output

Each Homework task will have its own section covering all of the topics above.

## Air Quality Index Data

### Highlights of Analysis 

### Dataset Description


### References to dataset

### Analysis

The necessary libraries for this task are the following

In [3]:
# read in pandas
import pandas as pd
# import matplotlib for colors
import matplotlib.pyplot as plt 